In [1]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA, make_loss
from functions import *
from scipy.linalg import hadamard

In [4]:
n_samples = 1000
n_test = 100
n = 10
p = 8
rng = np.random.RandomState(0)

fit_loss = 'l2'
reg = 'l1'

fit_function, der_function = {
                              'l2': (l2_fit, l2_der),
                              'logreg': (logreg_fit, logreg_der)
                              }[fit_loss]
reg_function, prox = {
                      'l2': (l2_pen, l2_prox),
                      'l1': (l1_pen, l1_prox),
                      None: (no_pen, no_prox)
                      }[reg]
loss = make_loss(fit_function, reg_function)


H = hadamard(p) / np.sqrt(p)
#S = p / np.arange(1, p + 1)
#S **= 2
S = np.ones(p)
K = np.dot(H, S[:, None] * H.T)

z_true = rng.randn(p, n_samples)
D = rng.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = rng.randn(n, n_samples)

z_test = rng.randn(p, n_test)
eps_test = rng.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if fit_loss == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = lbda_max * 0.1
print(X.shape)

(10, 1000)


In [5]:
ista = LISTA(D, lbda, 1000, fit_loss, reg)
print(loss(np.zeros((p, n_samples)), X, D, lbda))
f_star = loss(ista.transform(X), X, D, lbda)
print(f_star)

8.82852839123171
5.452118497263439


In [11]:
n_layers = 3
lista = LISTA(D, lbda, n_layers, fit_loss, reg, variables='both')
loss(lista.transform(X), X, D, lbda)

5.71450201082143

In [12]:
print(lista.weights[0][0, 0])
print(lista.weights[1][0, 0])

0.1275790310340033
0.34674165090767023


In [13]:
lista.fit(X, l_rate=1e-1, max_iter=1000, verbose=True)

it 0, loss = 5.715e+00, grad = 1.09e+00
it 100, loss = 5.551e+00, grad = 9.36e-02
it 200, loss = 5.547e+00, grad = 9.09e-02
it 300, loss = 5.543e+00, grad = 9.06e-02
it 400, loss = 5.540e+00, grad = 9.06e-02
it 500, loss = 5.538e+00, grad = 9.05e-02
it 600, loss = 5.536e+00, grad = 9.37e-02
it 700, loss = 5.534e+00, grad = 9.51e-02
it 800, loss = 5.532e+00, grad = 9.56e-02
it 900, loss = 5.531e+00, grad = 9.65e-02


In [14]:
print(lista.weights[0][0, 0])
print(lista.weights[1][0, 0])

0.04225529374501078
0.5612749779806477


In [15]:
lista_one = LISTA(D, lbda, 1, fit_loss, reg)
lista_one.fit(X, l_rate=1e-2, max_iter=1000, batch_size=1000, verbose=True)

it 0, loss = 6.351e+00, grad = 1.59e+00
it 100, loss = 5.762e+00, grad = 3.51e-01
it 200, loss = 5.705e+00, grad = 1.48e-01
it 300, loss = 5.693e+00, grad = 7.61e-02
it 400, loss = 5.689e+00, grad = 4.23e-02
it 500, loss = 5.688e+00, grad = 2.30e-02
it 600, loss = 5.688e+00, grad = 1.50e-02
it 700, loss = 5.688e+00, grad = 1.00e-02
it 800, loss = 5.688e+00, grad = 6.08e-03
it 900, loss = 5.688e+00, grad = 5.41e-03


In [16]:
f_star = loss(LISTA(D, lbda, 1000, fit_loss, reg).transform(X_test), X_test, D, lbda)
f0 = loss(np.zeros_like(z_test), X_test, D, lbda)

In [17]:
ista = LISTA(D, lbda, n_layers, fit_loss, reg)
print('Avg test loss using ISTA with %d iterations: %.2e' %
      (n_layers, (loss(ista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using ISTA with 3 iterations: 6.69e-02


In [18]:
print('Avg test loss using LISTA with %d layers: %.2e' %
      (n_layers, (loss(lista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using LISTA with 3 layers: 2.40e-02
